In [2]:
from bs4 import BeautifulSoup
import urllib2
import urllib
import json
import csv
import time
import pickle
import pandas as pd

In [151]:
html_pages = load_obj("angel_html_c_full")

In [ ]:
def backup(obj, iter_num):
    name = "fin_data_" + str(iter_num)
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
        
def load_obj(name):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [ ]:
request_headers = {"Host": "angel.co",
"Connection": "keep-alive",
"Content-Length": "18",
"Origin": "https://angel.co",
"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/46.0.2490.86 Safari/537.36",
"Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
"Accept": "*/*",
"X-Requested-With": "XMLHttpRequest",
"Referer": "https://angel.co/companies",
"Accept-Language": "en-US,en;q=0.8",
}


start = time.time()
#html_pages = {}
for page in range(3500):
    link = "https://angel.co/company_filters/search_data"
    data = urllib.urlencode({"page":str(page), "sort":"raised"})
    request = urllib2.Request(link, headers=request_headers)
    contents = urllib2.urlopen(request, data=data).read()
    content_d = json.loads(contents)
    beg = "https://angel.co/companies/startups?" + "&".join(["ids%5B%5D=" + str(id) for id in content_d['ids']])
    end = "&total=" + str(content_d["total"]) + "&page=" + str(content_d["page"]) + "&sort=" + str(content_d["sort"]) + "&new=" + str(content_d["new"]) + "&hexdigest=" + str(content_d["hexdigest"])
    link = beg + end
    html_pages[page] = urllib2.urlopen(link).read()
    
    time.sleep(1)
    if page % 100 == 0:
        mins = (time.time() - start) / 60
        print 'Percent progress ' + str(100 * float(page) / 3500) + ' running for ' + str(mins) + ' mins'
        backup(html_pages, page)

In [ ]:
request_headers = {"Host": "angel.co",
"Connection": "keep-alive",
#"Content-Length": "100",
"Origin": "https://angel.co",
"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/46.0.2490.86 Safari/537.36",
"Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
"Accept": "*/*",
"X-Requested-With": "XMLHttpRequest",
"Referer": "https://angel.co/companies",
"Accept-Language": "en-US,en;q=0.8",
"DNT": "1"}

start = time.time()
html_pages_c = []
vals = range(0, 100000000, 100000)
max_val = vals[1:]
min_val = vals[0:-1]
base_url = "https://angel.co/company_filters/search_data" 
print 'Starting'
for i in range(999):
    min_v, max_v = min_val[i], max_val[i]
    for page in range(20):
        data="filter_data%5Braised%5D%5Bmin%5D=" + str(min_v) + "&filter_data%5Braised%5D%5Bmax%5D=" + str(max_v) + "&sort=signal&page=" + str(page)
        request = urllib2.Request(base_url, headers=request_headers)
        contents = urllib2.urlopen(request, data=data).read()
        content_d = json.loads(contents)
        if content_d['ids'] != []:
            beg = "https://angel.co/companies/startups?" + "&".join(["ids%5B%5D=" + str(id) for id in content_d['ids']])
            end = "&total=" + str(content_d["total"]) + "&page=" + str(content_d["page"]) + "&sort=" + str(content_d["sort"]) + "&new=" + str(content_d["new"]) + "&hexdigest=" + str(content_d["hexdigest"])
            link = beg + end
            html_pages_c.append(urllib2.urlopen(link).read())
        else:
            break
        time.sleep(0.1)
    if i % 50 == 0:
        mins = (time.time() - start) / 60
        print 'Percent progress ' + str(100 * i / 50.0) + ' running for ' + str(mins) + ' mins'
        backup(html_pages_c, max_v)
backup(html_pages_c, "full")

In [152]:
# All possible fields

soup = BeautifulSoup(html_pages[1], "lxml")
soup.find_all("div", class_="pitch")[0]
soup.find_all("div", class_="value")[0]
soup.find_all("div", class_="market")[0]
soup.find_all("div", class_="website")[0]
soup.find_all("div", class_="joined")[0]
soup.find_all("div", class_="location")[0]
soup.find_all("div", class_="company_size")[0]
soup.find_all("div", class_="stage")[0]
soup.find_all("div", class_="raised")[0]

<div class="column hidden_column raised sortable" data-column="raised">\\nTotal Raised\\n</div>

In [155]:
len_pages = len(html_pages)
print 'Starting'
start = time.time()
names, raised, pitch, stage, market = [], [], [], [], []
for i in range(1, len_pages): 
    soup = BeautifulSoup(html_pages[i], "lxml")
    names_unf = soup.find_all("div", class_="name")
    raised_unf = soup.find_all("div", class_="raised")
    pitch_unf = soup.find_all("div", class_="pitch")
    stage_unf = soup.find_all("div", class_="stage")
    market_unf = soup.find_all("div", class_="market")
    if i % 100 == 0:
        mins = (time.time() - start) / 60
        print 'Percent progress ' + str(100 * float(i) / len_pages) + ' running for ' + str(mins) + ' mins'
    
    res_len = min(len(names_unf), len(raised_unf), len(pitch_unf), 
        len(stage_unf), len(market_unf))
    for j in range(res_len): 
        names.append(names_unf[j].text[2:-2])
        raised.append(raised_unf[j].text[22:-4])
        pitch.append(pitch_unf[j].text[2:-2])
        stage.append(stage_unf[j].text[15:-4])
        market.append(market_unf[j].text[16:-4])

startup_data_full = {"Names":names, "Raised": raised, "Pitch":pitch, "Stage":stage, "Market":market} 

Starting
Percent progress 3.34784064279 running for 0.221906415621 mins
Percent progress 6.69568128557 running for 0.435572417577 mins
Percent progress 10.0435219284 running for 0.655195017656 mins
Percent progress 13.3913625711 running for 0.875916063786 mins
Percent progress 16.7392032139 running for 1.09028981527 mins
Percent progress 20.0870438567 running for 1.30546389818 mins
Percent progress 23.4348844995 running for 1.5151494503 mins
Percent progress 26.7827251423 running for 1.7292738835 mins
Percent progress 30.1305657851 running for 1.9336484313 mins
Percent progress 33.4784064279 running for 2.13388544718 mins
Percent progress 36.8262470706 running for 2.34065020084 mins
Percent progress 40.1740877134 running for 2.5415830493 mins
Percent progress 43.5219283562 running for 2.73126256466 mins
Percent progress 46.869768999 running for 2.91019073327 mins
Percent progress 50.2176096418 running for 3.08468063275 mins
Percent progress 53.5654502846 running for 3.25285019875 mins


In [212]:
names = [name.replace(' ', '-') for name in names]
startup_data_full = {"Names":names, "Raised": raised, "Pitch":pitch, "Stage":stage, "Market":market} 
startup_df = pd.DataFrame(startup_data_full)
startup_df.head()

,Market,Names,Pitch,Raised,Stage
0,,Unwind-Me,"Massage Made Simple: The best therapists, stra...",,
1,Marketplaces,Jackson-Square-Ventures,VC firm investing in SaaS and marketplaces.,-,
2,SaaS,Contour-Ventures,,-,-
3,-,GSV-Capital,,-,-
4,Financial Services,Vast-Ventures,Investing to improve human potential,-,-


In [195]:
comp_data = {}
companies = list(startup_df['Names'].unique())
companies = [company.replace(' ', '-') for company in companies]
error_comps = []
companies_len = len(companies)
for i in range(companies_len):
    try:
        company = companies[i]
        link = "https://angel.co/" + company + "?utm_source=companies"
        soup = BeautifulSoup(urllib2.urlopen(link).read(), "lxml")
        fin_details = soup.find_all("div", class_="details")
        comp_data[company] = {"finance":[]}
        for fin_detail in fin_details:
            fin_round = fin_detail.find_all("div", class_="type")[0].text[1:-1]
            fin_date = fin_detail.find_all("div", class_ ="date_display")[0].text
            fin_amount = fin_detail.find_all("div", class_="raised")
            fin_amount = fin_amount[0].text[1:-1]
            comp_data[company]["finance"].append((fin_round, fin_date, fin_amount))
        prod_desc = soup.find_all("div", class_="product_desc")[0]
        comp_data[company]["desc"] = prod_desc.find_all("div", class_="content")[0].text[1: -1]
        if i % 1000 == 0:
            mins = (time.time() - start) / 60
            print 'Percent progress ' + str(100.0 * i / companies_len) + ' running for ' + str(mins) + ' mins'
            backup(comp_data, companies.index(company))
    except:
        error_comps.append(company)

Percent progress 0.0 running for 20.6441246311 mins
Percent progress 69.3200854948 running for 310.646753867 mins
Percent progress 92.4267806597 running for 405.472500134 mins


In [242]:
fin_data = load_obj('fin_data_full')

def fin_isvalid(data):
    stages = [dets[0] for dets in data]
    if any("Series B" in s for s in stages) or any("Series C" in s for s in stages):
        if stages.count('Series A') < 2 and stages.count('Series B') < 2 and stages.count('Series C') < 2 and stages.count('Series D') < 2:
                return True
    return False
name, fin, desc = [], [], []
for k in fin_data.keys():
    if fin_data[k].has_key('finance'):
        if fin_isvalid(fin_data[k]['finance']):
            name.append(k)
            fin.append(fin_data[k]['finance'])
            if fin_data[k].has_key('desc'):
                desc.append(fin_data[k]['desc'])
            else:
                desc.append('')   

no_stage, seed, series_a, series_b, series_c, series_d = [], [], [], [], [], []
for fin_list in fin:
    c_no_stage, c_seed, c_series_a, c_series_b, c_series_c, c_series_d = [], [], [], [], [], []
    for fin_round in fin_list:
        if fin_round[0] == 'Seed':
            c_seed.append(fin_round)
        elif fin_round[0] == 'No Stage':
            c_no_stage.append(fin_round)
        elif fin_round[0] == 'Series A':
            c_series_a.append(fin_round)
        elif fin_round[0] == 'Series B':
            c_series_b.append(fin_round)
        elif fin_round[0] == 'Series C':
            c_series_c.append(fin_round)
        elif fin_round[0] == 'Series D':
            c_series_d.append(fin_round)
    no_stage.append(c_no_stage)
    seed.append(c_seed)
    series_a.append(c_series_a)
    series_b.append(c_series_b)
    series_c.append(c_series_c)
    series_d.append(c_series_d)
for funding in [seed, no_stage, series_a, series_b, series_c, series_d]:
    for i in range(len(funding)):
        ele = funding[i]
        if ele == []:
            funding[i] = ('', '', '')
        else:
            funding[i] = ele[0]
            
seed_amount = [val[2] for val in seed]
no_stage_amount = [val[2] for val in no_stage]
series_a_amount = [val[2] for val in series_a]
series_b_amount = [val[2] for val in series_b]
series_c_amount = [val[2] for val in series_c]
series_d_amount = [val[2] for val in series_d]

seed_date = [val[1] for val in seed]
no_stage_date = [val[1] for val in no_stage]
series_a_date = [val[1] for val in series_a]
series_b_date = [val[1] for val in series_b]
series_c_date = [val[1] for val in series_c]
series_d_date = [val[1] for val in series_d]

market, pitch, stage = [], [], []
for comp in name:
    companydf = startup_df[startup_df["Names"] == comp]
    market.append(list(companydf.Market)[0])
    pitch.append(list(companydf.Pitch)[0])
    stage.append(list(companydf.Stage)[0])

funding_full = {"Names":name, "Market": market, "Pitch": pitch, "Stage": stage, "Description":desc,
                "No_Stage_Date": no_stage_date, "No_Stage_Amount":no_stage_amount,                
                "Seed_Date": seed_date, "Seed_Amount":seed_amount,
                "Series_A_Date": series_a_date, "Series_A_Amount":series_a_amount, 
                "Series_B_Date":series_b_date, "Series_B_Amount":series_b_amount, 
                "Series_C_Date":series_c_date, "Series_C_Amount":series_c_amount,
                "Series_D_Date":series_d_date, "Series_D_Amount":series_d_amount} 
funding_df = pd.DataFrame(funding_full)
funding_df = funding_df[funding_df["Stage"] != "Acquired"]
funding_df.head()

,Description,Market,Names,No_Stage_Amount,No_Stage_Date,Pitch,Seed_Amount,Seed_Date,Series_A_Amount,Series_A_Date,Series_B_Amount,Series_B_Date,Series_C_Amount,Series_C_Date,Series_D_Amount,Series_D_Date,Stage
1,,Cable,Epic-Sciences,,,,,,,,"$13,000,000","Nov 13, 2012","$30,000,000","Jul 30, 2014",,,Series C
2,,All Students,Apreso-Classroom,,,,,,,,"$15,000,000","Oct 14, 2008",,,,,Series B
3,Visualead (视觉码) creates better interactions be...,Bridging Online and Offline,Visualead,,,Effective and Secure Offline to Mobile experie...,"$750,000","Mar 25, 2012","$1,600,000","Aug 15, 2013",Unknown,"Jan 20, 2015",,,,,Series B
4,,Food Processing,Onshift,"$7,000,000","Feb 3, 2014",,,,,,"$3,000,000","Feb 2, 2012",,,,,Series C
5,,-,Xendex-Holding,,,,,,Unknown,"Jun 25, 2008","$3,500,000","Nov 30, 2009",,,,,Series A


In [3]:
funding_df=pd.read_csv("funding.csv")
funding_df.fillna('', inplace=True)
funding_df.head(10)

,Description,Market,Names,No_Stage_Amount,No_Stage_Date,Pitch,Seed_Amount,Seed_Date,Series_A_Amount,Series_A_Date,Series_B_Amount,Series_B_Date,Series_C_Amount,Series_C_Date,Series_D_Amount,Series_D_Date,Stage
0,,Cable,Epic-Sciences,,,,,,,,"$13,000,000","Nov 13, 2012","$30,000,000","Jul 30, 2014",,,Series C
1,,All Students,Apreso-Classroom,,,,,,,,"$15,000,000","Oct 14, 2008",,,,,Series B
2,Visualead (视觉码) creates better interactions be...,Bridging Online and Offline,Visualead,,,Effective and Secure Offline to Mobile experie...,"$750,000","Mar 25, 2012","$1,600,000","Aug 15, 2013",Unknown,"Jan 20, 2015",,,,,Series B
3,,Food Processing,Onshift,"$7,000,000","Feb 3, 2014",,,,,,"$3,000,000","Feb 2, 2012",,,,,Series C
4,,-,Xendex-Holding,,,,,,Unknown,"Jun 25, 2008","$3,500,000","Nov 30, 2009",,,,,Series A
5,Palo Alto-based FilmLoop has reportedly layed ...,Software,filmloop,,,,,,"$5,600,000","Feb 1, 2005","$7,000,000","May 1, 2006",,,,,Series A
6,,,Mochi-Media,,,,,,"$4,000,000","Mar 12, 2008","$10,000,000","Jun 18, 2008",,,,,
7,,-,SkyRecon-Systems,,,,,,"$3,730,000","Sep 12, 2005","$6,500,000","Mar 12, 2007",,,,,Series A
8,Challenge your brain with games designed by ne...,,Lumos-Labs,,,Creator of Lumosity,"$450,000","Jun 11, 2007","$3,100,000","Jun 3, 2008",,,"$32,500,000","Jun 16, 2011","$31,500,000","Aug 22, 2012",
9,"ROBLOX is a comprehensive 3D creation, publish...",Blockchains,ROBLOX,,,User-Generated Online Gaming Platform,,,"$2,200,000","Aug 14, 2009","$4,000,000","Jun 14, 2011",,,,,Series A
